## PySST basics tutorial

We will go over some of the basic functionality of PySST. Although not all functionality
is covered, this tutorial should get you familiar with the way PySST works. For a more
complete overview of functionality, see to the API reference.

First we point to the location of some test files that we will use in this tutorial:

In [8]:
import os
from pathlib import Path

import pysst

filepath = Path(os.getcwd())

local_folder = filepath
test_borehole_file = filepath.parent / "tests/data/test_boreholes.parquet"


### Loading and inspecting data

Data is loaded through various reader functions. In the API Reference you can find 
information on those functions and how to use them. In this example we will use the 
basic 'pysst.read_sst_cores' function to read parquet files of borehole data. You point
to the file (test_borehole_file, see previous code block) and specify the horizontal 
and vertical reference systems (more on those later).

In [9]:
borehole_collection = pysst.read_sst_cores(
                                            test_borehole_file, 
                                            vertical_reference="NAP", 
                                            horizontal_reference=28992
                                        )

PySST is programmed in an object-oriented style. In the above case 'borehole_collection'
is an instance of a so-called 'BoreholeCollection' object. A BoreholeCollection has 
various attributes that describe what is inside the object (e.g. the borehole data 
itself and all relevant metadata). In addition it has 'methods' that allow the user to
conduct various operations on the object.

Let's print 'borehole_collection' to see what it sais:

In [10]:
print(borehole_collection)

BoreholeCollection:
# header = 13


As you can see, 'borehole_collection' is of the type 'BoreholeCollection'. 
You can view 'BoreholeCollection' as the template of what a collection of boreholes
should look like (it is a Python class) and 'borehole_collection' as an instance of this
template filled in this case with 13 boreholes and associated attributes. Let's see what 
these attributes are...

The attribute 'header' is a table (pandas.DataFrame object) containing one line per 
borehole. It contains a minimal amount of required information for each borehole in the 
collection: the borehole ID ('nr'), x and y-coordinates, surface elevation ('mv'),
end depth and a point geometry (required for export to GIS etc.).

In [11]:
print(borehole_collection.header)

       nr       x       y    mv   end                       geometry
0    HB-6  127562  502816 -1.69 -5.59  POINT (127562.000 502816.000)
1   HB-11  128866  504391 -3.82 -8.62  POINT (128866.000 504391.000)
2    HB-8  128867  504421 -2.09 -5.99  POINT (128867.000 504421.000)
3   HB-81  128867  504414 -3.48 -8.58  POINT (128867.000 504414.000)
4   HB-12  129010  504400 -3.65 -8.95  POINT (129010.000 504400.000)
5   HB-13  129135  504414 -3.56 -8.46  POINT (129135.000 504414.000)
6   HB-04  127272  502323 -1.74 -5.54  POINT (127272.000 502323.000)
7   HB-10  129287  504474 -3.17 -7.97  POINT (129287.000 504474.000)
8   HB-14  129294  504441 -3.68 -9.58  POINT (129294.000 504441.000)
9   HB-09  129129  504447 -2.90 -8.00  POINT (129129.000 504447.000)
10  HB-01  127082  502096 -2.18 -5.88  POINT (127082.000 502096.000)
11  HB-02  127099  502088 -2.82 -6.62  POINT (127099.000 502088.000)
12  HB-03  127296  502338 -3.27 -6.67  POINT (127296.000 502338.000)


The 'data' attribute is also a pandas.DataFrame and contains lines for every layer in 
every borehole. It therefore contains the actual description of layers in the
boreholes as well as top and bottom elevations for each layer:

In [12]:
print(borehole_collection.data.head())

     nr       x       y    mv   end   top  bottom lith ad1  ad2   org   
0  HB-6  127562  502816 -1.69 -5.59 -1.69   -2.69    K  s1  NaN  None  \
1  HB-6  127562  502816 -1.69 -5.59 -2.69   -3.59    K  s1  NaN    H2   
2  HB-6  127562  502816 -1.69 -5.59 -3.59   -3.69    K  s1  NaN  None   
3  HB-6  127562  502816 -1.69 -5.59 -3.69   -3.79    K  s1  NaN  None   
4  HB-6  127562  502816 -1.69 -5.59 -3.79   -3.89    K  s1  NaN  None   

  lith_comb  pen     T  Prik  
0       Ks1  NaN   NaN   NaN  
1       Ks1  NaN   NaN   NaN  
2       Ks1  NaN  14.9  3.88  
3       Ks1  4.3  14.3  3.97  
4       Ks1  4.3  13.9  3.58  


As you can see there are additional columns for the top and bottom depths of layers as 
well as observations of lithology, admixture1, admixture2, organic material and some 
field measurements.

### Positional reference systems

The current vertical and horizontal reference systems used in the 'borehole_collection'
can be accessed through the 'vertical_reference' and 'horizontal_reference' attributes: 

In [13]:
print(borehole_collection.vertical_reference)
print(borehole_collection.horizontal_reference)

NAP
28992


The vertical reference can be one of three options:
- "NAP"           : Vertical positions are given in meters with respect to NAP
- "surfacelevel"  : The surface level is always 0. (e.g. tops can be 0, -1, -2, -3)
- "depth"         : Borehole length, starting at 0. (e.g. tops can be 0, 1, 2, 3)

Horizontal reference can be any coordinate reference system. The number returned above
(28992) is the EPSG number for the RD crs. See also https://epsg.io/

You can change the vertical reference in place. Now we will change it from NAP to depth:

In [14]:
# Change vertical reference to depth
borehole_collection.change_vertical_reference("depth")

# Show results
print(borehole_collection.vertical_reference)
print(borehole_collection.data.head())

# Change back to NAP for the rest of this tutorial
borehole_collection.change_vertical_reference("NAP")

depth
     nr       x       y    mv   end  top  bottom lith ad1  ad2   org   
0  HB-6  127562  502816 -1.69 -5.59 -0.0     1.0    K  s1  NaN  None  \
1  HB-6  127562  502816 -1.69 -5.59  1.0     1.9    K  s1  NaN    H2   
2  HB-6  127562  502816 -1.69 -5.59  1.9     2.0    K  s1  NaN  None   
3  HB-6  127562  502816 -1.69 -5.59  2.0     2.1    K  s1  NaN  None   
4  HB-6  127562  502816 -1.69 -5.59  2.1     2.2    K  s1  NaN  None   

  lith_comb  pen     T  Prik  
0       Ks1  NaN   NaN   NaN  
1       Ks1  NaN   NaN   NaN  
2       Ks1  NaN  14.9  3.88  
3       Ks1  4.3  14.3  3.97  
4       Ks1  4.3  13.9  3.58  


Note that the layer tops and bottoms are now increasing downward.

You can change the horizontal reference in a similar way:

In [15]:
# Convert to EPSG 32631 (UTM 31N)
borehole_collection.change_horizontal_reference(32631, only_geometries=False)

# Show results (note how the coordinates changed)
print(borehole_collection.horizontal_reference)
print(borehole_collection.header.head())

# Change back to Rijksdriehoek EPSG 28992
borehole_collection.change_horizontal_reference(28992, only_geometries=False)

32631
      nr              x             y    mv   end   
0   HB-6  634569.174062  5.819872e+06 -1.69 -5.59  \
1  HB-11  635820.288528  5.821489e+06 -3.82 -8.62   
2   HB-8  635820.296503  5.821519e+06 -2.09 -5.99   
3  HB-81  635820.527825  5.821512e+06 -3.48 -8.58   
4  HB-12  635963.898270  5.821503e+06 -3.65 -8.95   

                         geometry  
0  POINT (634569.174 5819872.270)  
1  POINT (635820.289 5821489.338)  
2  POINT (635820.297 5821519.352)  
3  POINT (635820.528 5821512.357)  
4  POINT (635963.898 5821503.091)  


If you would have used only_geometries=True, only the geometry column in the header 
would be updated. For loading later exports of this collection in GIS, it is sufficient
to only update the geometry column. The columns 'x' and 'y' are unaffected in this case.


### Creating selections and slices

There are several ways to make subsets of a collection, such as:

**Spatial selections**
- `select_within_bbox` - Select data points in the collection within a bounding box
- `select_with_points` - Select data points in the collection within distance from point geometries (a geopandas Dataframe)
- `select_with_lines` - Select data points in the collection within distance from line geometries (a geopandas Dataframe)
- `select_within_polygons` - Select data points in the collection within polygon geometries (a geopandas Dataframe)

**Conditional selections**
- `select_by_values` - Select data points in the collection based on the presence of certain values in one or more of the data columns
- `select_by_length` - Select data points in the collection based on length requirements 
- `select_by_depth` - Select data points in the collection based on depth constraints

**Slicing**
- `slice_depth_interval` - Slice boreholes in the collection down to the specified depth interval
- `slice_by_values` - Slice boreholes in the collection based on value (e.g. only sand layers, remove others)

We won't go over all of these. Refer to the API reference on how to use these methods.

Below we select boreholes within a simple bounding box, characterized by the xmin, xmax, ymin and ymax coordinates:

In [16]:
# Select boreholes between x = 128600 and 130000 and y = 504000 and 505000
selected_boreholes = borehole_collection.select_within_bbox(128600, 130000, 504000, 505000)

# Show result of selection
print(selected_boreholes)

BoreholeCollection:
# header = 8


We selected 8 boreholes in this newly created instance 'selected_boreholes', down from 
13 in the original 'borehole_collection'.

From this selection, we only want boreholes with sand. Main lithology is (for this set 
of boreholes) given in the column "lith". Sand is denoted by "Z" (Zand). We use the
'select_by_values' method in the following way:

In [17]:
# Select boreholes that contain sand (anywhere in one of the layers)
boreholes_with_sand = selected_boreholes.select_by_values("lith", "Z")

# Show result of selection, 7 boreholes contain sand in at least one of their layers
print(boreholes_with_sand)

BoreholeCollection:
# header = 7


7 out of 8 boreholes are in the newly created collection 'boreholes_with_sand'. You can
also use multiple values in conjunction with the 'how' argument to select boreholes that
contain one of the given lithologies or all of them at the same time:

In [18]:
# Select boreholes that contain sand OR clay (anywhere in one of the layers)
boreholes_with_sand_or_clay = selected_boreholes.select_by_values("lith", ["Z", "K"], how="or")

# Actually all boreholes have sand or clay as there are still 8 boreholes in the collection
print(boreholes_with_sand_or_clay)

# Select boreholes that contain sand AND clay at the same time (anywhere in one of the layers)
boreholes_with_sand_and_clay = selected_boreholes.select_by_values("lith", ["Z", "K"], how="and")

# 7/8 boreholes have both sand and clay in one of their layers
print(boreholes_with_sand_and_clay)


BoreholeCollection:
# header = 8
BoreholeCollection:
# header = 7


You can use select_by_length to select cores above a certain length:

In [19]:
# Select only boreholes between 5 and 6 meters long
boreholes_between_5_and_6_m = selected_boreholes.select_by_length(min_length=5, max_length=6)

# Show result of selection, only 4 out of 8 boreholes now remain.
print(boreholes_between_5_and_6_m.header)

      nr              x             y    mv   end   
3  HB-81  128867.000094  504414.00047 -3.48 -8.58  \
4  HB-12  129010.000094  504400.00047 -3.65 -8.95   
8  HB-14  129294.000094  504441.00047 -3.68 -9.58   
9  HB-09  129129.000094  504447.00047 -2.90 -8.00   

                        geometry  
3  POINT (128867.000 504414.000)  
4  POINT (129010.000 504400.000)  
8  POINT (129294.000 504441.000)  
9  POINT (129129.000 504447.000)  


c:\Users\onselen\AppData\Local\miniforge3\envs\pysst_env\lib\site-packages\geopandas\geodataframe.py:1475: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


We are now left with four boreholes. As you can tell from the surface level and end depth,
they are all between 5 and 6 m long.

### Basic analyses

There are a few basic methods to analyse borehole data. Currently implemented are:

**Generic**
- `get_area_labels` - Get specified labels of polygon geometries that points in the collection project on (e.g. add labels from the geological map to boreholes)
- `get_cumulative_layer_thickness` - Get the cumulative thickness of certain layers
- `get_layer_top` - Get top of certain layers

**Specific to BoreholeCollection**
- `cover_layer_thickness` - Algorithm to determine the (non-permeable) cover layer thickness

**Specific to CptCollection**
- `add_ic` - Compute and add non-normalized soil behaviour type index to measurements
- `add_lithology` - Add Robertson-Fugro classification to measurements

In the following example, we'd like to know the total thickness of layers with a strong 
organic admixture in the first 5 meters of our boreholes in the 'borehole_collection' 
that we created earlier. For this we have to combine a few methods to get the desired 
results:

In [20]:
# Since we want to know the thickness in the first 5 m with respect to surface level, 
# lets first normalize the boreholes vertically by using 'depth' as vertical reference:
borehole_collection.change_vertical_reference("depth")

# Now we can easily slice off the first 5 m as follows:
borehole_collection_sliced = borehole_collection.slice_depth_interval(upper_boundary=5)

# With only the first 5 m of our boreholes in 'borehole_collection_sliced', we can now
# get the cumulative thickness of layers that have 'H2' in the 'org' column. We will
# also include the new data in the header our borehole collection object (otherwise it
# only returns a dataframe with the results).
borehole_collection_sliced.get_cumulative_layer_thickness("org", "H2", include_in_header=True)

# Show results in the updated header, which now has a 'H2_thickness' column appended to it
print(borehole_collection_sliced.header)



       nr              x              y    mv   end   
0    HB-6  127562.000093  502816.000469 -1.69 -5.59  \
1   HB-11  128866.000094  504391.000470 -3.82 -8.62   
2    HB-8  128867.000094  504421.000470 -2.09 -5.99   
3   HB-81  128867.000094  504414.000470 -3.48 -8.58   
4   HB-12  129010.000094  504400.000470 -3.65 -8.95   
5   HB-13  129135.000094  504414.000470 -3.56 -8.46   
6   HB-04  127272.000092  502323.000469 -1.74 -5.54   
7   HB-10  129287.000094  504474.000470 -3.17 -7.97   
8   HB-14  129294.000094  504441.000470 -3.68 -9.58   
9   HB-09  129129.000094  504447.000470 -2.90 -8.00   
10  HB-01  127082.000092  502096.000469 -2.18 -5.88   
11  HB-02  127099.000092  502088.000469 -2.82 -6.62   
12  HB-03  127296.000092  502338.000469 -3.27 -6.67   

                         geometry  H2_thickness  
0   POINT (127562.000 502816.000)           0.9  
1   POINT (128866.000 504391.000)           0.0  
2   POINT (128867.000 504421.000)           0.0  
3   POINT (128867.000 504414.

As you can see, borehole HB-6 has a total thickness of strongly organic layers of 0.9 m,
whereas HB-10 and HB-14 have 10 cm of strongly organic material in the first 5 m below
the surface.

### Exporting data

A Borehole- or CptCollection can be exported to various formats to allow for further
analyses and visualisation in other software or for sharing data.

**Export layer data**
- `to_parquet` - Save to parquet file. Preferred method for sharing and re-using data in PySST. This output can be read by the `read_sst_cores` and `read_sst_cpt` functions.
- `to_csv` - Save to csv file.
- `to_ipf` - Save to iMod point file for use in iMod-QGIS or legacy iMod 5
- `to_vtm` - Save to vtm (multiple vtk) for 3D viewing 
- `to_datafusiontools` - Save to pickle or return object that can directly be used in Deltares DataFusionTools, see https://bitbucket.org/DeltaresGEO/datafusiontools/src/master/

**Export point geometries only**
- `to_shape` - Save point geometries to shapefile or geopackage. Fields added to the header during analysis will also be exported.
- `to_geoparquet` - Save point geometries to geoparquet. Fields added to the header during analysis will also be exported.